# DA5401: Assignment 4
**GMM-Based Synthetic Sampling for Imbalanced Data**

**Course:** DA5401  
**Topic:** Credit Card Fraud (highly imbalanced)  

* **Name:** Harshit Shukla 
* **Roll No:** DA25S003


**Documentation of Code:**

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
